In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_vix.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-2-41d669d0e6de>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
VIX,20210423,31.65,33.85,31.60,33.85,13639700,31.65,33.85,31.60,33.85,0,8000,23500,2021-04-23,16,2021,2021-16
VIX,20210422,29.50,31.65,29.20,31.65,17688000,29.50,31.65,29.20,31.65,0,500,4400,2021-04-22,16,2021,2021-16
VIX,20210420,29.65,30.10,29.25,29.60,4819700,29.65,30.10,29.25,29.60,0,1200,17100,2021-04-20,16,2021,2021-16


In [3]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(900)

In [10]:
#plot linechart for HPG price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=30, adjust=False).mean()

line_price = df['<Close>']
fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=700)
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})

fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='VIX'))



fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 30-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'VIX Violin chart',width=1200,height=700)


In [6]:
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'},width= 1200, height= 600)

In [7]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')


In [8]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [9]:
df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                        mode='lines', name='Rolling_volumn'))